<a href="https://colab.research.google.com/github/khalilbrikTN/Internship_AUCLibraryWebsite_Cross-APIBibliographicDataIntegration/blob/main/Group4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*italicized text*# Load Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install xlsxwriter
!pip install pymarc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import re

# Load Data (AUC library Data)



In [ ]:
batches_AUC = pd.read_csv('/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/barches_AUC.csv')

# Exploratory Data Analysis

# Load HathiTrust Data

In [ ]:
marc_xml_found_count = 0

# Helper Functions

def cleaner(value):
    """
    Extract and return the first sequence of digits from the input string.

    Parameters:
    value (str): The input string that may contain digits along with other characters.

    Returns:
    str or None: The first sequence of digits found in the string, or None if no digits are found.
    """
    # Use regular expression to find all sequences of digits in the string
    matches = re.findall(r'\d+', str(value))
    # Return the first sequence of digits found, or None if no digits are found
    return matches[0] if matches else value


def fetch_hathi_data(id_type, id_value):
    """
    Fetch data from HathiTrust API based on the given identifier type and value.

    Parameters:
    id_type (str): The type of identifier (e.g., 'oclc', 'lccn', 'isbn').
    id_value (str): The value of the identifier.

    Returns:
    dict: The JSON response from the HathiTrust API or None if the request fails.
    """
    url = f"https://catalog.hathitrust.org/api/volumes/full/{id_type}/{id_value}.json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def extract_record_data(record_data):
    """
    Extract relevant data from a HathiTrust record.

    Parameters:
    record_data (dict): The HathiTrust record data.

    Returns:
    tuple: A tuple containing the title and MARC XML of the record.
    """
    title_Hathi = record_data['titles'][0] if 'titles' in record_data else ''
    marc_xml = record_data['marc-xml'] if 'marc-xml' in record_data else ''
    return title_Hathi, marc_xml

def extract_item_data(item, record_id):
    """
    Extract relevant data from a HathiTrust item.

    Parameters:
    item (dict): The HathiTrust item data.
    record_id (str): The ID of the record the item belongs to.

    Returns:
    tuple: A tuple containing the view status and item URL of the record, or None if the item does not match the record ID.
    """
    if item['fromRecord'] == record_id:
        view_status = item['usRightsString']
        record_url = item['itemURL']
        return view_status, record_url
    return None

def create_result(AUC_title, title_Hathi, author, Publisher, datePublish, AUC_Number, cleaned_oclc, cleaned_lccn, cleaned_isbn, marc_xml, record_url, view_status, id_type, found_id):
    global marc_xml_found_count

    if marc_xml != '':
          marc_xml_found_count = marc_xml_found_count + 1

    return {
        'AUC_Title': AUC_title,
        'Hathi_Title': title_Hathi,
        'Author': author,
        'Published': Publisher,
        'D.O. Pub.': datePublish,
        'AUC no.': AUC_Number ,
        'OCLC': cleaned_oclc,
        'LCCN': cleaned_lccn,
        'ISBN': cleaned_isbn,
        'MARC XML': marc_xml,
        'Full View URL': record_url,
        'View': view_status,
        #'No ID provided' if not found_id else 'No matching record',
        'ID Used': id_type
    }


def process_identifiers(row, identifiers):

    found_id = False
    record_added = False

    AUC_title = row['TITLE']
    cleaned_oclc = identifiers.get('oclc')
    cleaned_lccn = identifiers.get('lccn')
    cleaned_isbn = identifiers.get('isbn')
    author = row['AUTHOR']
    Publisher = row['PUBLISHED']
    datePublish = row['D.O. Pub.']
    AUC_Number = row['AUC no.']


    for id_type, id_value in identifiers.items():
        if pd.notna(id_value) and id_value:
            found_id = True
            json_data = fetch_hathi_data(id_type, id_value)
            if json_data:
                for record_id, record_data in json_data['records'].items():
                    title_Hathi, marc_xml = extract_record_data(record_data)

                    for item in json_data['items']:
                        item_data = extract_item_data(item, record_id)
                        if item_data:
                            view_status, record_url = item_data

                            return create_result(AUC_title, title_Hathi, author, Publisher, datePublish, AUC_Number, cleaned_oclc, cleaned_lccn, cleaned_isbn, marc_xml, record_url, view_status, id_type, found_id)


    view_status = 'No ID provided' if not found_id else 'No matching record'

    return create_result(AUC_title, 'None', author, Publisher, datePublish, AUC_Number, cleaned_oclc, cleaned_lccn, cleaned_isbn, '', '', view_status, 'None', found_id)



def process_row(row):


    """
    Process a single row of the DataFrame to fetch and extract data from HathiTrust.

    Parameters:
    row (pd.Series): The original row of data.

    Returns:
    dict: A dictionary containing the result data.
    """
    identifiers = {
        'oclc': cleaner(row['OCLC no.']),
        'lccn': cleaner(row['LC no.']),
        'isbn': cleaner(row['ISBN'])
    }

    return process_identifiers(row, identifiers)



def process_dataframe(df, n):

    results = []
    row_num = 0
    for index, row in df.head(n).iterrows():
        new_row = process_row(row)
        results.append(new_row)
        row_num = row_num + 1

    return results

def save_results_to_csv(results_df, output_path):
    """
    Save the results to a CSV file.
    """

    results_df.to_csv(output_path, index=False)


def matching_stats(df):
    """
    Calculate and print matching statistics from the DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the result data.

    Returns:
    tuple: A tuple containing counts of empty MARC XML, empty View URLs, and ID usage statistics.
    """
    # Initialize counters
    empty_marc_count = 0
    found_marc_count = 0
    oclc_found_count = 0
    lccn_found_count = 0
    isbn_found_count = 0
    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        value_marc = row['MARC XML']
        if value_marc is None or value_marc == '':
            empty_marc_count += 1
        else:
            found_marc_count += 1
            if pd.notna(row['OCLC']):
                oclc_found_count += 1
            if pd.notna(row['LCCN']):
                lccn_found_count += 1
            if pd.notna(row['ISBN']):
                isbn_found_count += 1



    # Print the results
    print('Records Processed:', len(df))
    print('Records with NO MARC XML found:', empty_marc_count)
    print('Records with MARC XML found:', found_marc_count)
    print('Records with MARC XML found where OCLC is not empty:', oclc_found_count)
    print('Records with MARC XML found where lccn is not empty:', lccn_found_count)
    print('Records with MARC XML found where ISBN is not empty:', isbn_found_count)


# ***Batch1 : Process with Hathi***

In [ ]:
updatedBatch1 = pd.read_excel('/content/drive/MyDrive/GROUP 4 -- Metadata/Metadata Batches/NEW 18 7 2024/Metadata for Group 4-1.xlsx')

In [ ]:
sub_batches = []

sub_batches.append(updatedBatch1.iloc[0:10000])
sub_batches.append(updatedBatch1.iloc[10000:20000])
sub_batches.append(updatedBatch1.iloc[20000:30000])
sub_batches.append(updatedBatch1.iloc[30000:40000])
sub_batches.append(updatedBatch1.iloc[40000:50000])
sub_batches.append(updatedBatch1.iloc[50000:60001])


In [ ]:
counter = 1

for batch in sub_batches:

  HATHI_batch = pd.DataFrame(process_dataframe(batch, len(batch)))
  save_results_to_excel(HATHI_batch, f'/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/sub{counter}.xlsx')
  counter = counter + 1


### Concatenate results

In [ ]:
sub1 = pd.read_excel('/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/sub1.xlsx')
sub2 = pd.read_excel('/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/sub2.xlsx')
sub3 = pd.read_excel('/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/sub3.xlsx')
sub4 = pd.read_excel('/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/sub4.xlsx')
sub5 = pd.read_excel('/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/sub5.xlsx')
sub6 = pd.read_excel('/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/sub6.xlsx')

# Concatenate all DataFrames into a single DataFrame
concatenated_df = pd.concat([sub1, sub2, sub3, sub4, sub5, sub6], ignore_index=True)

# Save the concatenated DataFrame to a CSV file
output_path = '/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/concatenated_output.csv'
concatenated_df.to_csv(output_path, index=False)

# Experiment with Open Library

In [ ]:
def fetch_openlibrary_data(id_type, id_value):
    base_url = 'https://openlibrary.org/api/books'
    url = f'{base_url}?bibkeys={id_type.upper()}:{id_value}&format=json&jscmd=data'
    response = requests.get(url)
    if response.status_code == 200:
        print(response.json())
        return response.json()
    return None

def extract_openlibrary_record_data(record_data):
    title = record_data.get('title', 'None')
    view_link = record_data.get('url', None)


    preview_link = record_data.get('ebooks', None)


    identifiers = {
        'oclc': None,
        'lccn': None,
        'isbn': None
    }
    if 'identifiers' in record_data:
        identifiers['oclc'] = record_data['identifiers'].get('oclc', [None])[0]
        identifiers['lccn'] = record_data['identifiers'].get('lccn', [None])[0]
        identifiers['isbn'] = record_data['identifiers'].get('isbn_10', [None])[0] or record_data['identifiers'].get('isbn_13', [None])[0]

    return title, view_link, identifiers

def process_row_for_openlibrary(row):
    identifiers = {
        'oclc': cleaner(row['OCLC']),
        'lccn': cleaner(row['LCCN']),
        'isbn': cleaner(row['ISBN'])
    }

    AUC_title = row['AUC_Title']
    HATH_title = row['Hathi_Title']
    author = row['Author']
    Publisher = row['Published']
    datePublish = row['D.O. Pub.']
    AUC_Number = row['AUC no.']
    xml_HATHI = row['MARC XML']
    Full_View_URL_HATHI = row['Full View URL (HATHI)']
    View_HATHI= row['View (HATHI)']
    ID_HATHI = row['ID Used (HATHI)']
    openlibrary_title = 'None'
    view_link = ''
    found_id = False

    for id_type, id_value in identifiers.items():
        if pd.notna(id_value) and id_value:
            json_data = fetch_openlibrary_data(id_type, id_value)
            if json_data:
                key = f'{id_type.upper()}:{id_value}'
                if key in json_data:
                    record_data = json_data[key]
                    openlibrary_title, view_link, new_identifiers = extract_openlibrary_record_data(record_data)
                    for k, v in new_identifiers.items():
                        if not identifiers[k] and v:
                            identifiers[k] = v
                    found_id = True
                    break

    return {
        'AUC_Title': AUC_title,
        'Hathi_Title': HATH_title,
        'OpenLibrary_Title': openlibrary_title,
        'Author': author,
        'Published': Publisher,
        'D.O. Pub.': datePublish,
        'OCLC': identifiers['oclc'],
        'LCCN': identifiers['lccn'],
        'ISBN': identifiers['isbn'],
        'MARC XML(HATHI)' : xml_HATHI,
        'Full View URL (HATHI)' : Full_View_URL_HATHI,
        'View (HATHI)' : View_HATHI,
        'ID Used (HATHI)' : ID_HATHI,
        'View Link (OpenLibrary)': view_link,
        'Found_OpenLibrary': 'Yes' if found_id else 'No',
        'AUC no.': AUC_Number
    }

def process_dataframe(df, n):

    results = []
    row_num = 0
    for index, row in df.head(n).iterrows():
        new_row = process_row_for_openlibrary(row)
        results.append(new_row)
        row_num = row_num + 1

    return results

def save_results_to_excel(results_df, output_path):
    """
    Save the results to an Excel file.
    """

    results_df.to_excel(output_path, index=False, engine='xlsxwriter')

In [ ]:
HATHI_OUT_1 = pd.read_csv('/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/BATCH1OUT/HATHIOUT/concatenated_output.csv')

In [ ]:
len(HATHI_OUT_1)

60000

In [ ]:
df_combined = pd.DataFrame(process_dataframe(HATHI_OUT_1, 10))

https://archive.org/details/lostmenofamerica00holb
None
None
None
https://archive.org/details/twelvestoriesbyf0000unse
None
None
None


In [ ]:
df_combined.head()

df_combined.to_excel('/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/tmp.xlsx', index=False)


In [ ]:
sub_batches = []

sub_batches.append(HATHI_OUT_1.iloc[0:10000])
sub_batches.append(HATHI_OUT_1.iloc[10000:20000])
sub_batches.append(HATHI_OUT_1.iloc[20000:30000])
sub_batches.append(HATHI_OUT_1.iloc[30000:40000])
sub_batches.append(HATHI_OUT_1.iloc[40000:50000])
sub_batches.append(HATHI_OUT_1.iloc[50000:60001])

In [ ]:
counter = 1

for batch in sub_batches:

  OPEN_BATCH = pd.DataFrame(process_dataframe(batch, len(batch)))
  save_results_to_excel(OPEN_BATCH, f'/content/drive/MyDrive/GROUP 4 -- Metadata/Codes/Output/BATCH1OUT/OPENOUT/sub{counter}.xlsx')
  counter = counter + 1
  print(counter)
